In [1]:
model_type = 'SVM'
from sklearn.svm import SVC

In [2]:
import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import ray
from ray import tune
from ray.tune.schedulers import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
import ConfigSpace as CS
from functools import partial

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from joblib import dump, load

In [3]:
# Device configuration
device = torch.device('cpu')
classes = ['Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

In [4]:
def model_eval(features, labels, model, dataset):
    actual_classes = labels
    predicted_classes = model.predict(features)
    
    #since labels are read in based on ordering in the folder, 
    #this corrects the labels so they reflect the correct classes
    #############################################################
    if dataset == 'gen':
        labels_dict = {
            0:3,
            1:5,
            2:7,
            3:8
        }
        for index, item in enumerate(actual_classes):
            actual_classes[index] = labels_dict[item]
    #############################################################
    
    performance_report = classification_report(
                        actual_classes, 
                        predicted_classes, 
                        labels=list(range(0,10)), 
                        target_names=classes, 
                        output_dict=True
                        )

    with open(f'performance_report_{dataset}.json', 'w') as f:
        json.dump(performance_report, f, indent=0)

    overall_accuracy = accuracy_score(actual_classes, predicted_classes)

    comparison_list = [['Actual', 'Predicted']]

    for i in range(0,len(actual_classes)):
        comparison_list.append([actual_classes[i], predicted_classes[i]])
    np.savetxt(f'class_pred_{dataset}.csv', comparison_list, delimiter=',', fmt='%s')
    
    labeled_actual = []
    labeled_predicted = []
    for index, item in enumerate(actual_classes):
        labeled_actual.append(classes[actual_classes[index]])
        labeled_predicted.append(classes[predicted_classes[index]])
        
    plt.clf()
    c_matrix = confusion_matrix(labeled_actual, labeled_predicted)
    c_df = pd.DataFrame(c_matrix, index=classes, columns=classes)
    plt.figure(figsize=(13,13))
    sns.heatmap(c_df, annot=True, fmt='g')
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Class')
    plt.xlabel('Predicted Class')
    plt.savefig(f'labeled_confusion_matrix_{dataset}.png', bbox_inches='tight')
    plt.clf()
    
    return overall_accuracy

In [5]:
xtest = torch.load('C:/Users/s_kal/Desktop/9039-ML/Final Project/Code/test_extracted_features.pt',map_location=torch.device('cpu'))
ytest = torch.load('C:/Users/s_kal/Desktop/9039-ML/Final Project/Code/test_extracted_labels.pt',map_location=torch.device('cpu'))
xgen = torch.load('C:/Users/s_kal/Desktop/9039-ML/Final Project/Code/gen_extracted_features.pt',map_location=torch.device('cpu'))
ygen = torch.load('C:/Users/s_kal/Desktop/9039-ML/Final Project/Code/gen_extracted_labels.pt',map_location=torch.device('cpu'))

model = load('C:/Users/s_kal/Desktop/9039-ML/Final Project/Code/Evaluate Optimal Models/SVM Results/model_SVM.joblib')

#     For learning curve of best model:
#     plt.clf()
#     plot_learning_curves(xtrain, ytrain, xtest, ytest, model)
#     plt.savefig('learning_curve.png')

test_acc =  model_eval(xtest, ytest, model, 'test')
gen_acc = model_eval(xgen, ygen, model, 'gen')

C:\Users\s_kal\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\s_kal\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\s_kal\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<Figure size 432x288 with 0 Axes>

<Figure size 936x936 with 0 Axes>

<Figure size 936x936 with 0 Axes>

In [8]:
test_acc

0.9761

In [9]:
gen_acc

0.8719893798596624